In [13]:
import pandas as pd
import datalad as dl
from datalad import api as dlapi
import json

In [4]:
ds = dlapi.Dataset('/Users/nielsond/data/datalad/datasets.datalad.org/')

In [ ]:
! cd {ds}
! datalad update --merge -r

In [5]:
res = !datalad -f json -c datalad.search.index-egrep-documenttype=all search -d /Users/nielsond/data/datalad/datasets.datalad.org/  path:.*\.nii.gz

In [10]:
jres = []
json_start = False
for rr in res:
    # Wait for the first string that has a curly bracket in it
    if json_start:
        rd = json.loads(rr)
        for mdk, mdv in rd['metadata'].items():
            for innerk, innerv in mdv.items():
                rd['metadata__' + mdk + '__' + innerk]=innerv
        jres.append(rd)
    elif '{' in rr:
        json_start = True

In [14]:
dl_df = pd.DataFrame(jres)

In [15]:
# Datasts
dl_df.path.str.split('/').str[6].unique()

array(['abide', 'adhd200', 'corr', 'crcns', 'dbic', 'dicoms', 'hbnssi',
       'indi', 'labs', 'openfmri', 'openneuro', 'workshops'], dtype=object)

In [16]:
# Figure out which datasets have hashes
dl_df['hash'] = dl_df.loc[dl_df.metadata__annex__key.notnull(),'metadata__annex__key'].str.split('--').str[-1].str.split('.').str[0]


In [17]:
dl_df[dl_df.hash.notnull()].path.str.split('/').str[6].unique()

array(['crcns', 'dbic', 'dicoms', 'indi', 'labs', 'openfmri', 'openneuro',
       'workshops'], dtype=object)

In [18]:
dl_df.to_csv('datalad_metadata.csv', index=False)

In [19]:
!rsync -avcH datalad_metadata.csv prom:/data/nielsond/mriqc/mriqc_tsne/
    

********************************** WARNING - WARNING - WARNING ***************************************
|                                                                                                    |
|     This warning banner provides privacy and security notices consistent with applicable           |
|     federal laws, directives, and other federal guidance for accessing this Government system,     |
|     which includes (1) this computer network, (2) all computers connected to this network,         |
|     and (3) all devices and storage media attached to this network or to a computer on this        |
|     network.                                                                                       |
|     - This system is provided for Government-authorized use only.                                  |
|     - Unauthorized or improper use of this system is prohibited and may result in                  |
|       disciplinary action and/or civil and criminal penalties.         

In [57]:
!rsync -avch rot:/roentgenium_data/mriqc/mriqc_tsne/*_datalad_and_mriqc*.csv .

 ********************************* WARNING - WARNING - WARNING **************************************
 |                                                                                                   |
 |      You are accessing a U.S. Government information system, WARNINGhich includes                 |
 |      (1) this computer, (2) this computerputer network, (3) all computers connected to            |
 |      this network, and (4) all devices and storage media attached to this                         |
 |      network or to a computer on this network.  This information system is provided               |
 |      for U.S. Government-authorized use only. Unauthorized or  improper use of this               |
 |      system may result in disciplinary action, as well as civil                                   |
 |      and criminal penalties.                                                                      |
 |                                                                        